# A Case Study: The Effect of Gun Ownership on Gun-Homicide Rates

We consider the problem of estimating the effect of gun
ownership on the homicide rate. For this purpose, we estimate the following partially
linear model

$$
 Y_{j,t} = \beta D_{j,(t-1)} + g(Z_{j,t}) + \epsilon_{j,t}.
$$

## Data

$Y_{j,t}$ is log homicide rate in county $j$ at time $t$, $D_{j, t-1}$ is log  fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership,  and  $Z_{j,t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. The parameter $\beta$ is the effect of gun ownership on the
homicide rates, controlling for county-level demographic and economic characteristics. 

The sample covers 195 large United States counties between the years 1980 through 1999, giving us 3900 observations.

In [3]:
# import Pkg; Pkg.add("StatsModels")

In [1]:
using Pkg, CSV, DataFrames, StatsModels, GLM, Random

In [79]:
data = CSV.File("../data/gun_clean.csv") |> DataFrame;
println("Number of rows: ", size(data, 1))
println("Number of columns: ", size(data, 2))

Number of rows: 3900
Number of columns: 415


### Preprocessing

To account for heterogeneity across counties and time trends in  all variables, we remove from them county-specific and time-specific effects in the following preprocessing.

In [ ]:
#################################  Find Variable Names from Dataset ########################


function varlist(df = nothing , type_dataframe = ["numeric","categorical","string"], pattern=String , exclude =  nothing)

    varrs = []
    if "numeric" in type_dataframe
        append!(varrs, [i for i in names(data) if eltype(eachcol(data)[i]) <: Number])    
    end
    if "categorical" in type_dataframe
        append!(varrs,[i for i in names(data) if eltype(eachcol(data)[i]) <: CategoricalVector])
    end
    if "string" in type_dataframe
        append!(varrs,[i for i in names(data) if eltype(eachcol(data)[i]) <: String])
    end
    varrs[(!varrs in exclude) & varrs[findall(x->contains(x,pattern),names(data))]]
end

In [80]:
################################# Create Variables ###############################

# Dummy Variables for Year and County Fixed Effects
fixed = filter(x -> contains(x, "X_Jfips"), names(data))
year = filter(x -> contains(x, "X_Tyear"), names(data))
println("Variable: <<Fixed>> has ", length(fixed), " features")
println("Variable: <<Year>> has ", length(year), " outcomes")

Variable: <<Fixed>> has 195 features
Variable: <<Year>> has 21 outcomes


In [81]:
# Census Control Variables
census = []
census_var = ["AGE", "BN", "BP", "BZ", "ED", "EL", "HI", "HS", "INC", "LF", "LN", "PI", "PO", "PP", "PV", "SPR", "VS"]

for i in 1:size(census_var, 1) 
    append!(census, filter(x -> contains(x, census_var[i]), names(data)))
end

println("Variable: <<Census>> has ", length(census), " features")

Variable: <<Census>> has 186 features


In [82]:
################################ Variables ##################################

# Treatment Variable
d = ["logfssl"]

# Outcome Variable
y = ["logghomr"]

# Other Control Variables
X1 = ["logrobr", "logburg", "burg_missing", "robrate_missing"]
X2 = ["newblack", "newfhh", "newmove", "newdens", "newmal"]

variable = [y, d, X1, X2, census]
varlis = []

for i in variable
    append!(varlis, i)
end

In [3]:
#################################  Partial out Fixed Effects ########################

# Running the following lines takes aprox. 10 minutes (depends on your CPU)

rdata = DataFrame(CountyCode = data[:, "CountyCode"]);

for i in 1:size(varlis, 1)
    rdata[!, varlis[i]] = residuals(lm(term(Symbol(varlis[i])) ~ sum(term.(Symbol.(year))) + sum(term.(Symbol.(fixed))), data))
end

first(rdata, 6)

Variable: <<Fixed>> has 195 features
Variable: <<Year>> has 21 outcomes
Variable: <<Census>> has 186 features


,CountyCode,logghomr,logfssl,logrobr,logburg,burg_missing,robrate_missing
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1073,-0.134778,0.0961271,0.150893,-0.124395,0.0104613,-0.021229
2,1073,-0.239622,0.0808094,0.0401683,-0.134781,0.0104613,-0.0194181
3,1073,-0.0786772,0.0573399,-0.017679,-0.167909,0.0104613,-0.0220374
4,1073,-0.331465,0.0816945,-0.00963344,-0.22925,0.0104613,-0.0194181
5,1073,-0.31664,0.0253655,-0.0267151,-0.176635,0.00324793,-0.0208037
6,1073,0.105132,-0.00677726,-0.151487,-0.189069,0.0104613,0.016953


In [83]:
# load dataset
rdata_read = CSV.File("../data/gun_clean2.csv") |> DataFrame
data = rdata_read[!, names(rdata)]
n = size(data,1)

3900

## We check that our results are equal to R results at 6 decimals

In [84]:
column_names = names(data)
result = []

for i in 1:size(data,1)
    for j in 1:size(data,2)
        data[i,j] = round(data[i,j], digits=6)
        rdata[i,j] = round(rdata[i,j], digits=6)
    end
end

for col in column_names
    result = sum(data[!,col] .== rdata[!,col])

    if result .== 3900
        println("Column ", col,  " are equal at 6 decimals")
    else
        println("Column ", col,  " are not equal at 6 decimals")
    end

end

Column CountyCode are equal at 6 decimals
Column logghomr are equal at 6 decimals
Column logfssl are equal at 6 decimals
Column logrobr are equal at 6 decimals
Column logburg are equal at 6 decimals
Column burg_missing are equal at 6 decimals
Column robrate_missing are equal at 6 decimals
Column newblack are equal at 6 decimals
Column newfhh are equal at 6 decimals
Column newmove are equal at 6 decimals
Column newdens are equal at 6 decimals
Column newmal are equal at 6 decimals
Column AGE010D are equal at 6 decimals
Column AGE050D are equal at 6 decimals
Column AGE110D are equal at 6 decimals
Column AGE170D are equal at 6 decimals
Column AGE180D are equal at 6 decimals
Column AGE270D are equal at 6 decimals
Column AGE310D are equal at 6 decimals
Column AGE320D are equal at 6 decimals
Column AGE350D are equal at 6 decimals
Column AGE380D are equal at 6 decimals
Column AGE410D are equal at 6 decimals
Column AGE470D are equal at 6 decimals
Column AGE570D are equal at 6 decimals
Column AG

Now, we can construct the treatment variable, the outcome variable and the matrix $Z$ that includes the control variables.

In [91]:
# Treatment Variable
D = rdata[!, d]
println("Variable: <<D>> has ", size(D), " outcomes")

# Treatment Variable
D = rdata[!, d]

# Outcome Variable
Y = rdata[!, y]
println("Variable: <<Y>> has ", size(Y), " outcomes")

# Construct matrix Z
Z = rdata[!, varlis[3:end]]
println("Variable: <<Z>> has ", size(Z), " outcomes")

Variable: <<D>> has (3900, 1) outcomes
Variable: <<Y>> has (3900, 1) outcomes
Variable: <<Z>> has (3900, 195) outcomes


We have in total 195 control variables. The control variables $Z_{j,t}$ are from the U.S. Census Bureau and  contain demographic and economic characteristics of the counties such as  the age distribution, the income distribution, crime rates, federal spending, home ownership rates, house prices, educational attainment, voting paterns, employment statistics, and migration rates. 

In [86]:
clu = select(rdata, :CountyCode) # for clustering the standard errors
data = hcat(clu, Y, D, Z)
first(data, 6)

,CountyCode,logghomr,logfssl,logrobr,logburg,burg_missing,robrate_missing
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1073,-0.134778,0.096127,0.150893,-0.124395,0.010461,-0.021229
2,1073,-0.239622,0.080809,0.040168,-0.134781,0.010461,-0.019418
3,1073,-0.078677,0.05734,-0.017679,-0.167909,0.010461,-0.022037
4,1073,-0.331465,0.081694,-0.009633,-0.22925,0.010461,-0.019418
5,1073,-0.31664,0.025366,-0.026715,-0.176635,0.003248,-0.020804
6,1073,0.105132,-0.006777,-0.151487,-0.189069,0.010461,0.016953


In [87]:
size(data), size(rdata)

((3900, 198), (3900, 198))

In [25]:
# CSV.write("../data/gun_clean2.csv", data)

## The effect of gun ownership


### OLS

After preprocessing the data, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls as a baseline model.

In [28]:
# Run this line to avoid all the lines of code above
# data = CSV.File("../data/gun_clean2.csv") |> DataFrame

In [10]:
using FixedEffectModels

In [11]:
# OLS clustering at the County level

fm_1 = @formula(logghomr ~ 0 + logfssl + fe(CountyCode))
baseline_ols = reg(data, fm_1, Vcov.cluster(:CountyCode))

                        Fixed Effect Model                        
Number of obs:              3900  Degrees of freedom:            2
R2:                        0.006  R2 Adjusted:               0.006
F-Stat:                  18.9732  p-value:                   0.000
R2 within:                 0.006  Iterations:                    1
logghomr | Estimate Std.Error t value Pr(>|t|) Lower 95% Upper 95%
------------------------------------------------------------------
logfssl  | 0.282304 0.0648108 4.35582    0.000  0.155238   0.40937


In [12]:
println("2.5% : ", coeftable(baseline_ols).cols[5])
println("97.5% : " , coeftable(baseline_ols).cols[6])
println("Estimate: ", coeftable(baseline_ols).cols[1])
println("Cluster s.e. : " , r2(baseline_ols))
println("T-value : ", coeftable(baseline_ols).cols[3])
println("Pr(>|t|) : " , coeftable(baseline_ols).cols[4])

2.5% : [0.15523789539597105]
97.5% : [0.4093704387686366]
Estimate: [0.2823041670823038]
Cluster s.e. : 0.006193251272214595
T-value : [4.355820406592104]
Pr(>|t|) : [1.3597928318408193e-5]


The point estimate is $0.282$ with the confidence interval ranging from 0.167 to 0.397. This
suggests that increases in gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by 0.28%, without controlling for counties' characteristics.

Since our goal is to estimate the effect of gun ownership after controlling for a rich set county characteristics we next include the controls. First, we estimate the model by ols and then by an array of the modern regression methods using the double machine learning approach.

In [92]:
# define the variables
y = ["logghomr"]

data_columns = names(data)
no_relev_col = ["logfssl", "CountyCode", "logghomr"]

# This gives us: new_list = ['carrot' , 'lemon']
z = data[!, Not(no_relev_col)]

control_formula = term(:logghomr) ~ term(:logfssl) + sum(term.(names(z)[1:end]))
control_ols = reg(data, control_formula, Vcov.cluster(:CountyCode))

┌ Warning: estimated covariance matrix of moment conditions not of full rank.
│                  model tests should be interpreted with caution.
└ @ Vcov C:\Users\sandr\.julia\packages\Vcov\8Fkqk\src\utils.jl:5


                                   Linear Model                                   
Number of obs:                      3900   Degrees of freedom:                  181
R2:                                0.203   R2 Adjusted:                       0.164
F-Stat:                       2.71826e18   p-value:                           0.000
logghomr        |    Estimate   Std.Error      t value Pr(>|t|) Lower 95% Upper 95%
-----------------------------------------------------------------------------------
logfssl         |    0.190671 0.000214096      890.584    0.000  0.190251   0.19109
logrobr         |     0.18903     9.74947    0.0193888    0.985  -18.9258   19.3039
logburg         |    0.219294     4.47891    0.0489614    0.961  -8.56206   9.00065
burg_missing    |      1.5298     7.52835     0.203205    0.839  -13.2303   16.2899
robrate_missing |     1.13297     7.26053     0.156044    0.876  -13.1021    15.368
newblack        |    -4.34682     6.62062    -0.656558    0.512  -17.3272   8

In [51]:
# control_ols= reg(data,term(:logghomr) ~  term(:logfssl) + sum(term.(Symbol.(names(Z)))) + fe(:CountyCode))
# est_ols = coef(control_ols)[1]
# println(confint(control_ols)[1,1:2])
# println(est_ols[1])

In [14]:
println("For <<logfssl>> variable: ")
println("2.5% : ", coeftable(control_ols).cols[5][1])
println("97.5% : " , coeftable(control_ols).cols[6][1])
println("Estimate: ", coeftable(control_ols).cols[1][1])
println("Cluster s.e. : " , r2(control_ols))
println("T-value : ", coeftable(control_ols).cols[3][1])
println("Pr(>|t|) : " , coeftable(control_ols).cols[4][1])

For <<logfssl>> variable: 
2.5% : 0.19025077455652542
97.5% : 0.19109028864460129
Estimate: 0.19067053160056335
Cluster s.e. : 0.20296178713656843
T-value : 890.5843124451466
Pr(>|t|) : 0.0


# DML algorithm

Here we perform inference of the predictive coefficient $\beta$ in our partially linear statistical model, 

$$
Y = D\beta + g(Z) + \epsilon, \quad E (\epsilon | D, Z) = 0,
$$

using the **double machine learning** approach. 

For $\tilde Y = Y- E(Y|Z)$ and $\tilde D= D- E(D|Z)$, we can write
$$
\tilde Y = \alpha \tilde D + \epsilon, \quad E (\epsilon |\tilde D) =0.
$$

Using cross-fitting, we employ modern regression methods
to build estimators $\hat \ell(Z)$ and $\hat m(Z)$ of $\ell(Z):=E(Y|Z)$ and $m(Z):=E(D|Z)$ to obtain the estimates of the residualized quantities:

$$
\tilde Y_i = Y_i  - \hat \ell (Z_i),   \quad \tilde D_i = D_i - \hat m(Z_i), \quad \text{ for each } i = 1,\dots,n.
$$

Finally, using ordinary least squares of $\tilde Y_i$ on $\tilde D_i$, we obtain the 
estimate of $\beta$.

In [52]:
# import Pkg; Pkg.add("MLDataUtils")
# import Pkg; Pkg.add("MLBase")

In [15]:
using  MLDataUtils, MLBase

In [16]:
function DML2_for_PLM(z, d, y, dreg, yreg, nfold, clu)
    
    # Number of observations
    nobser = size(z, 1) 
    
    # Define folds indices 
    foldid = collect(Kfold(size(z)[1], nfold))
    
    # Create array to save errors 
    ytil = ones(nobser)
    dtil = ones(nobser)
    println("Folds: " )
    
    # loop to save results
    for i in 1:nfold
        
        # Lasso regression, excluding folds selected 
        dfit = dreg(z[foldid[i],:], d[foldid[i]])
        yfit = yreg(z[foldid[i],:], y[foldid[i]])
        
        # predict estimates using the 
        dhat = Lasso.predict(dfit, z[Not(foldid[i]),:])
        yhat = Lasso.predict(yfit, z[Not(foldid[i]),:])
        
        # save errors 
        dtil[Not(foldid[i])]   = (d[Not(foldid[i])] - dhat)
        ytil[Not(foldid[i])]   = (y[Not(foldid[i])] - yhat)
        println(i)
    end
    
    # Create dataframe
    data = DataFrame(ytil = ytil, dtil = dtil, clu = clu)
    
    # OLS clustering at the County level
    rfit = reg(data, @formula(ytil ~ dtil + fe(clu)))
    coef_est = coef(rfit)[1]
    se = FixedEffectModels.coeftable(rfit).cols[2]

    println(" coef (se) = ", coef_est ,"(",se,")")
    
    return rfit;
    
end

DML2_for_PLM (generic function with 1 method)

Now, we apply the Double Machine Learning (DML) approach with different machine learning methods. First, we load the relevant libraries.

Let us, construct the input matrices.

In [93]:
z = Matrix(Z);
d = D[!, 1];
y = Y[!, 1];
clu = rdata[!, :CountyCode];
first(DataFrame(logghomr = y, logfssl = d, CountyCode = clu ), 6)

,logghomr,logfssl,CountyCode
,Float64,Float64,Int64
1,-0.134778,0.096127,1073
2,-0.239622,0.080809,1073
3,-0.078677,0.05734,1073
4,-0.331465,0.081694,1073
5,-0.31664,0.025366,1073
6,0.105132,-0.006777,1073


In the following, we apply the DML approach with the differnt versions of lasso.


## Lasso

In [18]:
using Lasso

In [19]:
Random.seed!(123)

function dreg(z, d)
    fit(LassoModel, z, d, standardize = false)
end

############################

function yreg(z, y)
    fit(LassoModel, z, y,  standardize = false)
end

DML2_lasso = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)

Folds: 
1
2
3
4
5
6
7
8
9
10
 coef (se) = 0.21585868256187365([0.05692379809111925])


                      Fixed Effect Model                      
Number of obs:            3900  Degrees of freedom:        196
R2:                      0.004  R2 Adjusted:            -0.048
F-Stat:                14.3797  p-value:                 0.000
R2 within:               0.004  Iterations:                  1
ytil | Estimate Std.Error t value Pr(>|t|) Lower 95% Upper 95%
--------------------------------------------------------------
dtil | 0.215859 0.0569238 3.79206    0.000  0.104254  0.327464


## GLMnet

In [20]:
using GLMNet

In [21]:
function DML2_lasso_cv(z , d , y, dreg , yreg , nfold, clu)
    # Number of observations
    nobser = size(z, 1) 
    
    # Define folds indices 
    foldid = collect(Kfold(size(z)[1], nfold))
    
    # Create array to save errors 
    ytil = ones(nobser)
    dtil = ones(nobser)
    println("Folds: " )
    
    # loop to save results
    for i in 1:nfold
        
        # Lasso regression, excluding folds selected 
        dfit = dreg(z[foldid[i],:], d[foldid[i]])
        yfit = yreg(z[foldid[i],:], y[foldid[i]])
        
        # predict estimates using the 
        dhat = GLMNet.predict(dfit, z[Not(foldid[i]),:])
        yhat = GLMNet.predict(yfit, z[Not(foldid[i]),:])
        
        # save errors 
        dtil[Not(foldid[i])]   = (d[Not(foldid[i])] - dhat)
        ytil[Not(foldid[i])]   = (y[Not(foldid[i])] - yhat)
        println(i)
    end
    
    # Create dataframe
    data = DataFrame(ytil = ytil, dtil = dtil, clu = clu)
    
    # OLS clustering at the County level
    rfit = reg(data, @formula(ytil ~ dtil + fe(clu)))
    coef_est = coef(rfit)[1]
    se = FixedEffectModels.coeftable(rfit).cols[2]

    println(" coef (se) = ", coef_est ,"(",se,")")
    
    return rfit;
    
end

DML2_lasso_cv (generic function with 1 method)

### ML method = Cross-Validated Lasso from glmnet 

In [22]:
dreg(z, d) = glmnetcv(z, d, alpha = 1)    
yreg(z, y) = glmnetcv(z, y, alpha = 1)    

DML2_lasso_cv_1 = DML2_lasso_cv(z, d, y, dreg, yreg, 2,clu)

Folds: 
1
2
 coef (se) = 0.20275011053055503([0.057294865710519595])


                      Fixed Effect Model                      
Number of obs:            3900  Degrees of freedom:        196
R2:                      0.005  R2 Adjusted:            -0.048
F-Stat:                12.5225  p-value:                 0.000
R2 within:               0.003  Iterations:                  1
ytil | Estimate Std.Error t value Pr(>|t|) Lower 95% Upper 95%
--------------------------------------------------------------
dtil |  0.20275 0.0572949 3.53871    0.000 0.0904175  0.315083


### ML method = Elastic net from glmnet 

In [23]:
dreg(z,d) = glmnetcv(z,d, alpha = 0.5) 
yreg(z,y) = glmnetcv(z,y, alpha = 0.5)

DML2_elnet = DML2_lasso_cv(z, d, y, dreg, yreg, 10, clu)

Folds: 
1
2
3
4
5
6
7
8
9
10
 coef (se) = 0.2172461515632935([0.05691865677555617])


                      Fixed Effect Model                      
Number of obs:            3900  Degrees of freedom:        196
R2:                      0.005  R2 Adjusted:            -0.048
F-Stat:                14.5678  p-value:                 0.000
R2 within:               0.004  Iterations:                  1
ytil | Estimate Std.Error t value Pr(>|t|) Lower 95% Upper 95%
--------------------------------------------------------------
dtil | 0.217246 0.0569187 3.81678    0.000  0.105651  0.328841


### ML method = Ridge from glmnet 

In [24]:
dreg(z,d) = glmnetcv(z,d, alpha = 0) 
yreg(z,y) = glmnetcv(z,y, alpha = 0)

DML2_ridge = DML2_lasso_cv(z, d, y, dreg, yreg, 10, clu)

Folds: 
1
2
3
4
5
6
7
8
9
10
 coef (se) = 0.2149895024571906([0.05709057028411709])


                      Fixed Effect Model                      
Number of obs:            3900  Degrees of freedom:        196
R2:                      0.004  R2 Adjusted:            -0.048
F-Stat:                 14.181  p-value:                 0.000
R2 within:               0.004  Iterations:                  1
ytil | Estimate Std.Error t value Pr(>|t|) Lower 95% Upper 95%
--------------------------------------------------------------
dtil |  0.21499 0.0570906 3.76576    0.000  0.103057  0.326922


Here we also compute DML with OLS used as the ML method

In [26]:
Random.seed!(123)
function dreg(z,d)
    lm(z,d)
end

function yreg(z,y)
    lm(z,y)
end

DML2_ols = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)

Folds: 
1
2
3
4
5
6
7
8
9
10
 coef (se) = 0.19028171422895446([0.05593504593523386])


                      Fixed Effect Model                      
Number of obs:            3900  Degrees of freedom:        196
R2:                      0.004  R2 Adjusted:            -0.049
F-Stat:                11.5725  p-value:                 0.001
R2 within:               0.003  Iterations:                  1
ytil | Estimate Std.Error t value Pr(>|t|) Lower 95% Upper 95%
--------------------------------------------------------------
dtil | 0.190282  0.055935 3.40183    0.001 0.0806152  0.299948


Next, we also apply Random Forest for comparison purposes.

### Random Forest


In [162]:
# import Pkg; Pkg.add("MLJ")

In [27]:
using DecisionTree, MLJ

In [89]:
function DML2_RF(z , d , y, dreg , yreg , nfold, clu)
    
    # Number of observations
    nobser = size(z, 1) 
    
    # Define folds indices 
    foldid = collect(Kfold(size(z)[1], nfold))
    
    # Create array to save errors 
    ytil = ones(nobser)
    dtil = ones(nobser)
    println("Folds: " )
    
    # loop to save results
    for i in 1:nfold
        
        # Lasso regression, excluding folds selected 
        dfit = dreg(z[foldid[i],:], d[foldid[i]])
        yfit = yreg(z[foldid[i],:], y[foldid[i]])
        
        # predict estimates using the 
        dhat = apply_forest(dfit, z[Not(foldid[i]),:])
        yhat = apply_forest(yfit, z[Not(foldid[i]),:])
        
        # save errors 
        dtil[Not(foldid[i])]   = (d[Not(foldid[i])] - dhat)
        ytil[Not(foldid[i])]   = (y[Not(foldid[i])] - yhat)
        
        println(i)
        
    end
    
    # Create dataframe
    data = DataFrame(ytil = ytil, dtil = dtil, clu = clu)
    
    # OLS clustering at the County level
    rfit = reg(data, @formula(ytil ~ dtil + fe(clu)))
    coef_est = coef(rfit)[1]
    se = FixedEffectModels.coeftable(rfit).cols[2]

    println(" coef (se) = ", coef_est ,"(",se,")")
    
    return rfit;
  
end

DML2_RF (generic function with 1 method)

In [94]:
function dreg(z,d)
    RFmodel = build_forest(d,z)
end

function yreg(z,y)
    RFmodel = build_forest(y,z)
end

DML2_RF_1 = DML2_RF(z, d, y, dreg, yreg, 2, clu)

Folds: 
1
2


LoadError: "Some observations for the dependent variable are infinite"

We conclude that the gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by about 0.20% controlling for counties' characteristics.

Finally, let's see which method is actually better. We compute RMSE for predicting D and Y, and see which
of the methods works better.

In [117]:
    coef_est = coef(rfit)[1]
    se = FixedEffectModels.coeftable(rfit).cols[2]

LoadError: "Some observations for the dependent variable are infinite"

In [295]:
mods = [DML2_ols, DML2_lasso, DML2_lasso_cv_1, DML2_ridge, DML2_elnet]
mods_name = ["DML2_ols", "DML2_lasso", "DML2_lasso_cv", "DML2_ridge}", "DML2_elnet"]

function mdl(model, model_name)
    RMSEY = 

LoadError: syntax: incomplete: premature end of input

In [298]:
names(DML2_ols)

LoadError: MethodError: no method matching names(::Nothing)
[0mClosest candidates are:
[0m  names([91m::Union{DataFrames.DataFrameColumns, DataFrames.DataFrameRows}[39m) at C:\Users\sandr\.julia\packages\DataFrames\MA4YO\src\abstractdataframe\iteration.jl:261
[0m  names([91m::Union{DataFrames.DataFrameColumns, DataFrames.DataFrameRows}[39m, [91m::Any[39m) at C:\Users\sandr\.julia\packages\DataFrames\MA4YO\src\abstractdataframe\iteration.jl:262
[0m  names([91m::DataFrames.SubIndex[39m) at C:\Users\sandr\.julia\packages\DataFrames\MA4YO\src\other\index.jl:529
[0m  ...